In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

#view a sample of the data
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


## Local Government Area Summary

In [2]:
#Calculate the summary information required
unique_school_names = school_data_complete["school_name"].unique()
total_schools = len(unique_school_names)
total_students = school_data_complete["Student ID"].count()
unique_budget_figures = school_data_complete["budget"].unique()
total_budget = unique_budget_figures.sum()
avg_math_score = school_data_complete["maths_score"].mean()
avg_read_score = school_data_complete["reading_score"].mean()
per_pass_math = len(school_data_complete.loc[school_data_complete["maths_score"] >= 50, :])/total_students*100
per_pass_read = len(school_data_complete.loc[school_data_complete["reading_score"] >= 50, :])/total_students*100
num_pass_overall = len(school_data_complete.loc[(school_data_complete["reading_score"] >= 50) & (school_data_complete["maths_score"] >= 50), :])
per_pass_overall = num_pass_overall/total_students*100

#Create data frame with the results
LGA_Summary = pd.DataFrame({
    "Total Schools":[total_schools],
    "Total Students":[total_students],
    "Total Budget":[total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score":[avg_read_score],
    "% Passing Maths":[per_pass_math],
    "% Passing Reading":[per_pass_read],
    "% Overall Passing":[per_pass_overall]
})
#Clean up the format of the results
LGA_Summary["Total Students"] = LGA_Summary["Total Students"].map('{:,.0f}'.format)
LGA_Summary["Total Budget"] = LGA_Summary["Total Budget"].map('${:,.0f}'.format)
LGA_Summary["Average Math Score"] = LGA_Summary["Average Math Score"].map('{:,.2f}%'.format)
LGA_Summary["Average Reading Score"] = LGA_Summary["Average Reading Score"].map('{:,.2f}%'.format)
LGA_Summary["% Passing Maths"] = LGA_Summary["% Passing Maths"].map('{:,.2f}%'.format)
LGA_Summary["% Passing Reading"] = LGA_Summary["% Passing Reading"].map('{:,.2f}%'.format)
LGA_Summary["% Overall Passing"] = LGA_Summary["% Overall Passing"].map('{:,.2f}%'.format)

LGA_Summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",70.34%,69.98%,86.08%,84.43%,72.81%


## School Summary

In [3]:
#Take columns from 
school_summary = school_data[["school_name","type","size","budget"]]
school_summary = school_summary.rename(columns={"school_name":"School Name",
                     "type":"School Type",
                     "size":"Total Students",
                     "budget":"Total School Budget"})
school_summary.set_index("School Name", inplace=True)
school_summary.sort_index(ascending=True, inplace=True)

#Calculate statistics for each school
school_summary["Average Maths Score"] = school_data_complete.groupby(["school_name"])["maths_score"].mean()
school_summary["Average Reading Score"] = school_data_complete.groupby(["school_name"])["reading_score"].mean()
school_summary["Per Student Budget"]=school_summary["Total School Budget"]/school_summary["Total Students"]
school_summary["% Passing Math"] = school_data_complete.groupby(["school_name"]).apply(lambda x:x[x["maths_score"] >= 50]["maths_score"].count())/school_summary["Total Students"]*100
school_summary["% Passing Reading"] = school_data_complete.groupby(["school_name"]).apply(lambda x:x[x["reading_score"] >= 50]["reading_score"].count())/school_summary["Total Students"]*100
school_summary["% Overall Passing"] = school_summary["% Passing Math"]*school_summary["% Passing Reading"]/100

#Reorganise the headers
school_summary_present=school_summary[["School Type","Total Students","Total School Budget","Per Student Budget",
                               "Average Maths Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]

#Clean up the format of the results
school_summary_present["Total Students"] = school_summary_present["Total Students"].map('{:,.0f}'.format)
school_summary_present["Total School Budget"] = school_summary_present["Total School Budget"].map('${:,.0f}'.format)
school_summary_present["Per Student Budget"] = school_summary_present["Per Student Budget"].map('{:,.2f}'.format)
school_summary_present["Average Maths Score"] = school_summary_present["Average Maths Score"].map('{:,.2f}%'.format)
school_summary_present["Average Reading Score"] = school_summary_present["Average Reading Score"].map('{:,.2f}%'.format)
school_summary_present["% Passing Math"] = school_summary_present["% Passing Math"].map('{:,.2f}%'.format)
school_summary_present["% Passing Reading"] = school_summary_present["% Passing Reading"].map('{:,.2f}%'.format)
school_summary_present["% Overall Passing"] = school_summary_present["% Overall Passing"].map('{:,.2f}%'.format)

school_summary_present

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,Government,"4,976","$3,124,928",628.00,72.35%,71.01%,91.64%,87.38%,80.07%
Cabrera High School,Independent,"1,858","$1,081,356",582.00,71.66%,71.36%,90.85%,89.07%,80.92%
Figueroa High School,Government,"2,949","$1,884,411",639.00,68.70%,69.08%,81.65%,82.81%,67.62%
Ford High School,Government,"2,739","$1,763,916",644.00,69.09%,69.57%,82.44%,82.22%,67.78%
Griffin High School,Independent,"1,468","$917,500",625.00,71.79%,71.25%,91.21%,88.49%,80.71%
Hernandez High School,Government,"4,635","$3,022,020",652.00,68.87%,69.19%,80.95%,81.88%,66.28%
Holden High School,Independent,427,"$248,087",581.00,72.58%,71.66%,89.93%,88.52%,79.61%
Huang High School,Government,"2,917","$1,910,635",655.00,68.94%,68.91%,81.69%,81.45%,66.54%
Johnson High School,Government,"4,761","$3,094,650",650.00,68.84%,69.04%,82.06%,81.98%,67.27%


## Top Performing Schools (By % Overall Passing)

In [4]:
top_schools = school_summary.sort_values("% Overall Passing",ascending=False)
top_schools.head()

,School Type,Total Students,Total School Budget,Average Maths Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Independent,1858,1081356,71.657158,71.359526,582.0,90.850377,89.074273,80.924313
Griffin High School,Independent,1468,917500,71.788147,71.245232,625.0,91.212534,88.487738,80.711909
Bailey High School,Government,4976,3124928,72.352894,71.008842,628.0,91.639871,87.379421,80.074389
Holden High School,Independent,427,248087,72.583138,71.660422,581.0,89.929742,88.524590,79.609936
Wright High School,Independent,1800,1049400,72.047222,70.969444,583.0,91.777778,86.666667,79.540741


From the above data it seems that Independent schools are more likely to be have students that are performing well. This is also shown in the data below where Government schools are more likely to have less students with a passing mark.

## Bottom Performing Schools (By % Overall Passing)

In [5]:
bottom_schools = school_summary.sort_values("% Overall Passing",ascending=True)
bottom_schools.head()

,School Type,Total Students,Total School Budget,Average Maths Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Hernandez High School,Government,4635,3022020,68.874865,69.186408,652.0,80.949299,81.877023,66.278876
Huang High School,Government,2917,1910635,68.935207,68.910525,655.0,81.693521,81.453548,66.542271
Johnson High School,Government,4761,3094650,68.843100,69.039277,650.0,82.062592,81.978576,67.273744
Wilson High School,Independent,2283,1319574,69.170828,68.876916,578.0,82.785808,81.296540,67.301997
Figueroa High School,Government,2949,1884411,68.698542,69.077993,639.0,81.654798,82.807731,67.616486


## Maths Scores by Year

In [6]:
#Create a series for each Years math scores by using loc and a conditional to limit scores in those years
year_nine = school_data_complete.loc[school_data_complete["year"] == 9]
year_nine_maths = year_nine[["school_name","maths_score"]]
year_nine_avg_maths = year_nine_maths.groupby(["school_name"])["maths_score"].mean()
year_ten = school_data_complete.loc[school_data_complete["year"] == 10]
year_ten_maths = year_ten[["school_name","maths_score"]]
year_ten_avg_maths = year_ten_maths.groupby(["school_name"])["maths_score"].mean()
year_eleven = school_data_complete.loc[school_data_complete["year"] == 11]
year_eleven_maths = year_eleven[["school_name","maths_score"]]
year_eleven_avg_maths = year_eleven_maths.groupby(["school_name"])["maths_score"].mean()
year_twelve = school_data_complete.loc[school_data_complete["year"] == 12]
year_twelve_maths = year_twelve[["school_name","maths_score"]]
year_twelve_avg_maths = year_twelve_maths.groupby(["school_name"])["maths_score"].mean()

#Bring the series together & renaming headers
avg_maths_by_year = pd.merge(year_nine_avg_maths,year_ten_avg_maths,how='left',on="school_name")
avg_maths_by_year = avg_maths_by_year.rename(columns={"maths_score_x":"Year 9",
                                                     "maths_score_y":"Year 10"})
avg_maths_by_year = pd.merge(avg_maths_by_year,year_eleven_avg_maths,how='left',on="school_name")
avg_maths_by_year = avg_maths_by_year.rename(columns={"maths_score":"Year 11",})
avg_maths_by_year = pd.merge(avg_maths_by_year,year_twelve_avg_maths,how='left',on="school_name")
avg_maths_by_year = avg_maths_by_year.rename(columns={"maths_score":"Year 12",})

#Update the format of the columns
avg_maths_by_year["Year 9"] = avg_maths_by_year["Year 9"].map('{:,.2f}%'.format)
avg_maths_by_year["Year 10"] = avg_maths_by_year["Year 10"].map('{:,.2f}%'.format)
avg_maths_by_year["Year 11"] = avg_maths_by_year["Year 11"].map('{:,.2f}%'.format)
avg_maths_by_year["Year 12"] = avg_maths_by_year["Year 12"].map('{:,.2f}%'.format)

avg_maths_by_year

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,72.49%,71.90%,72.37%,72.68%
Cabrera High School,72.32%,72.44%,71.01%,70.60%
Figueroa High School,68.48%,68.33%,68.81%,69.33%
Ford High School,69.02%,69.39%,69.25%,68.62%
Griffin High School,72.79%,71.09%,71.69%,71.47%
Hernandez High School,68.59%,68.87%,69.15%,68.99%
Holden High School,70.54%,75.11%,71.64%,73.41%
Huang High School,69.08%,68.53%,69.43%,68.64%
Johnson High School,69.47%,67.99%,68.64%,69.29%


## Reading Score by Year

In [23]:
#Create a series for each Years math scores by using loc and a conditional to limit scores in those years
year_nine_read = year_nine[["school_name","reading_score"]]
year_nine_avg_read = year_nine_read.groupby(["school_name"])["reading_score"].mean()
year_ten_read = year_ten[["school_name","reading_score"]]
year_ten_avg_read = year_ten_read.groupby(["school_name"])["reading_score"].mean()
year_eleven_read = year_eleven[["school_name","reading_score"]]
year_eleven_avg_read = year_eleven_read.groupby(["school_name"])["reading_score"].mean()
year_twelve_read = year_twelve[["school_name","reading_score"]]
year_twelve_avg_read = year_twelve_read.groupby(["school_name"])["reading_score"].mean()

#Bring the series together & renaming headers
avg_read_by_year = pd.merge(year_nine_avg_read,year_ten_avg_read,how='left',on="school_name")
avg_read_by_year = avg_read_by_year.rename(columns={"reading_score_x":"Year 9",
                                                     "reading_score_y":"Year 10"})
avg_read_by_year = pd.merge(avg_read_by_year,year_eleven_avg_read,how='left',on="school_name")
avg_read_by_year = avg_read_by_year.rename(columns={"reading_score":"Year 11",})
avg_read_by_year = pd.merge(avg_read_by_year,year_twelve_avg_read,how='left',on="school_name")
avg_read_by_year = avg_read_by_year.rename(columns={"reading_score":"Year 12",})

#Update the format of the columns
avg_read_by_year["Year 9"] = avg_read_by_year["Year 9"].map('{:,.2f}%'.format)
avg_read_by_year["Year 10"] = avg_read_by_year["Year 10"].map('{:,.2f}%'.format)
avg_read_by_year["Year 11"] = avg_read_by_year["Year 11"].map('{:,.2f}%'.format)
avg_read_by_year["Year 12"] = avg_read_by_year["Year 12"].map('{:,.2f}%'.format)

avg_read_by_year

,Year 9,Year 10,Year 11,Year 12
school_name,,,,
Bailey High School,70.90%,70.85%,70.32%,72.20%
Cabrera High School,71.17%,71.33%,71.20%,71.86%
Figueroa High School,70.26%,67.68%,69.15%,69.08%
Ford High School,69.62%,68.99%,70.74%,68.85%
Griffin High School,72.03%,70.75%,72.39%,69.43%
Hernandez High School,68.48%,70.62%,68.42%,69.24%
Holden High School,71.60%,71.10%,73.31%,70.48%
Huang High School,68.67%,69.52%,68.74%,68.67%
Johnson High School,68.72%,69.30%,69.97%,67.99%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [12]:
#Create bins and labels for school spending
bins = [0,585.99,630.99,645.99,680]
labels=["<$585","$585-630","$630-645","$645-680"]

#Cut the data by the bins and labels
school_summary["Spending Category"] = pd.cut(school_summary["Per Student Budget"],bins,labels=labels,include_lowest=True)

#Take only the useful headings
summary_by_spending = school_summary[["Spending Category","Average Maths Score","Average Reading Score",
                                           "% Passing Math","% Passing Reading","% Overall Passing"]]
#Group by Spending Category
summary_by_spending = summary_by_spending.groupby("Spending Category")

summary_by_spending.mean()

,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Category,,,,,
<$585,71.364587,70.716577,88.835926,86.390517,76.844247
$585-630,72.065868,71.031297,91.518824,87.292423,79.887791
$630-645,69.854807,69.838814,84.686139,83.763585,71.009935
$645-680,68.884391,69.045403,81.568470,81.769716,66.698297


From the Spending data above there is no link between more spending in a school and higher percentage of passing students. Compared to the data below which has a strong correlation between smaller schools having a larger percentage of students passing.

## Scores by School Size

In [14]:
#Create bins and labels for school spending
bins = [0,999,1999,5000]
labels=["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]

#Cut the data by the bins and labels
school_summary["Size Category"] = pd.cut(school_summary["Total Students"],bins,labels=labels,include_lowest=True)

#Take only the useful headings
summary_by_size = school_summary[["Size Category","Average Maths Score","Average Reading Score",
                                           "% Passing Math","% Passing Reading","% Overall Passing"]]
#Group by Spending Category
summary_by_size = summary_by_size.groupby("Size Category")

summary_by_size.mean()

,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Size Category,,,,,
Small(<1000),72.335748,71.636864,90.806867,87.557513,79.499752
Medium(1000-2000),71.421650,70.720164,89.846560,86.714149,77.968028
Large(2000-5000),69.751809,69.576052,84.252804,83.301185,70.277892


## Scores by School Type

* Perform the same operations as above, based on school type

In [15]:
#Take only the useful headings
summary_by_type = school_summary[["School Type","Average Maths Score","Average Reading Score",
                                           "% Passing Math","% Passing Reading","% Overall Passing"]]
#Group by Spending Category
summary_by_type = summary_by_type.groupby("School Type")

summary_by_type.mean()

,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.703020
Independent,71.368822,70.718933,89.204043,86.247789,77.017705


As evidenced in the Top and Bottom data Independent Schools have a higher percentage of overall passing students.